In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent=dataset["Purchased"]

In [9]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [10]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [11]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [12]:
from sklearn.naive_bayes import BernoulliNB

from sklearn.model_selection import GridSearchCV

param_grid = {
     'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
    'binarize': [None, 0.0, 0.1, 0.5, 1.0],
    'fit_prior': [True, False],
    'class_prior': [None],
    'force_alpha': [True, False]
    }

grid = GridSearchCV(BernoulliNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
                         'binarize': [None, 0.0, 0.1, 0.5, 1.0],
                         'class_prior': [None], 'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.1, 'binarize': 0.0, 'class_prior': None, 'fit_prior': False, 'force_alpha': True}: 0.4942225312442918


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[40 45]
 [24 25]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.62      0.47      0.54        85
           1       0.36      0.51      0.42        49

    accuracy                           0.49       134
   macro avg       0.49      0.49      0.48       134
weighted avg       0.53      0.49      0.49       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.49039615846338536

In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008993,0.001535,0.029507,0.002158,0.1,None,None,True,True,"{'alpha': 0.1, 'binarize': None, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
1,0.010220,0.003055,0.024953,0.012617,0.1,None,None,True,False,"{'alpha': 0.1, 'binarize': None, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
2,0.008182,0.000819,0.011292,0.001498,0.1,None,None,False,True,"{'alpha': 0.1, 'binarize': None, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
3,0.008173,0.002035,0.010373,0.001619,0.1,None,None,False,False,"{'alpha': 0.1, 'binarize': None, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
4,0.009329,0.001826,0.009814,0.000677,0.1,0.0,None,True,True,"{'alpha': 0.1, 'binarize': 0.0, 'class_prior':...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.007786,0.000710,0.008606,0.000482,10.0,0.5,None,False,False,"{'alpha': 10.0, 'binarize': 0.5, 'class_prior'...",0.509839,0.539529,0.482156,0.50141,0.555303,0.517647,0.026386,19
96,0.006927,0.001434,0.010992,0.001546,10.0,1.0,None,True,True,"{'alpha': 10.0, 'binarize': 1.0, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
97,0.007085,0.000807,0.010404,0.001723,10.0,1.0,None,True,False,"{'alpha': 10.0, 'binarize': 1.0, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31
98,0.006956,0.000164,0.009529,0.000382,10.0,1.0,None,False,True,"{'alpha': 10.0, 'binarize': 1.0, 'class_prior'...",0.509779,0.525300,0.501410,0.50141,0.501410,0.507862,0.009302,31


In [20]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age:54
Salary:20000
Sex Male 0 or 1:1


In [21]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[0]


C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but BernoulliNB was fitted with feature names
  warnings.warn(
